In [1]:
using CSV
using DataFrames
using Serialization
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/md_data_2020_12_01.csv", missingstring="null"));

In [4]:
hospitals = sort(unique(rawdata[!,"Facility Name"]));
get_hospital_id(h) = h |> hash |> string |> (x -> "h" * x[end-7:end]);
convert_date(d) = Date(d, dateformat"m/d/y") + Year(2000);

data = select(rawdata,
    "Facility Name" => :hospital,
    "Facility Name" => ByRow(get_hospital_id) => :hospital_id,

    "COVID-19 Patients in Acute Care Beds" => :acute_occupancy,
    "COVID-19 Patients in ICU" => :icu_occupancy,
    ["COVID-19 Patients in Acute Care Beds", "COVID-19 Patients in ICU"] => ByRow((a,b) -> a+b) => :total_occupancy,
);
sort!(data, :hospital);

In [5]:
function latest_val(xs)
    xs = filter(x -> !ismissing(x), xs)
    z = length(xs) == 0 ? missing : xs[end]
    z = coalesce(z, 0)
    return z
end;

In [6]:
data_latest = combine(groupby(data, :hospital), [
    :hospital_id => latest_val => :hospital_id,
    :total_occupancy => latest_val => :total_occupancy,
    :icu_occupancy => latest_val => :icu_occupancy,
    :acute_occupancy => latest_val => :acute_occupancy,
]);

In [7]:
capacity_rawdata = DataFrame(CSV.File("../data/capacity.csv"));

In [8]:
covid_capacity_pct = 0.4;

In [9]:
capacity_data = select(capacity_rawdata,
    :hospital,
    :capacity_icu => (x -> x .* covid_capacity_pct) => :icu_beds,
    :capacity_acute => (x -> x .* covid_capacity_pct) => :acute_beds,
    :capacity_allbeds => (x -> x .* covid_capacity_pct) => :total_beds,
);

In [10]:
data_latest = rightjoin(data_latest, capacity_data, on=:hospital);

In [11]:
compute_load(a,b) = (a==0) ? 0.0 : (b==0) ? 1.0 : a/b;

In [12]:
data_latest.total_load = compute_load.(data_latest.total_occupancy, data_latest.total_beds);
data_latest.icu_load = compute_load.(data_latest.icu_occupancy, data_latest.icu_beds);
data_latest.acute_load = compute_load.(data_latest.acute_occupancy, data_latest.acute_beds);

In [13]:
metadata = DataFrame(CSV.File("../data/hospital_meta.csv"));
filter!(row -> row.in_forecast, metadata);

In [14]:
data_combined = rightjoin(data_latest, metadata, on=:hospital, makeunique=true);
select!(data_combined, Not(:hospital_id_1));

In [15]:
sort!(data_combined, :hospital);

In [16]:
data_combined

,hospital,hospital_id,total_occupancy,icu_occupancy,acute_occupancy,icu_beds,acute_beds,total_beds,total_load,icu_load,acute_load,in_forecast,address,county,healthcare_region,city,zipcode,lat,long,place_id,state,state_abbrev,hsa_id,hsa_name,hrr_id,hrr_name
,String?,String?,Int64?,Int64?,Int64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Bool,String,String,String,String,Int64?,Float64,Float64,String,String,String,Int64?,String?,Int64?,String?
1,Anne Arundel Medical Center,h39740093,75,10,65,16.0,113.6,129.6,0.578704,0.625,0.572183,1,"2002 Medical Pkwy, Annapolis, MD 21401, USA",Anne Arundel County,Region III,Annapolis,21401,38.9904,-76.5374,ChIJwQbdhNf2t4kR9wSFLYAT0Go,Maryland,MD,21001,"Annapolis, MD",113,"Washington, DC"
2,Atlantic General Hospital,h92616877,5,0,5,3.6,17.2,20.4,0.245098,0.0,0.290698,1,"9733 Healthway Dr, Berlin, MD 21811, USA",Worcester County,Region IV,Berlin,21811,38.3396,-75.2115,ChIJrTSS5gInuYkRtT2krn-1-Jw,Maryland,MD,21029,"Salisbury, MD",225,"Salisbury, MD"
3,Baltimore Washington Medical Center,h75227743,49,17,32,12.0,82.4,94.4,0.519068,1.41667,0.38835,1,"301 Hospital Dr, Glen Burnie, MD 21061, USA",Anne Arundel County,Region III,Glen Burnie,21061,39.1379,-76.6218,ChIJofV1-bz8t4kRUG1GNyt_7cA,Maryland,MD,21016,"Glen Burnie, MD",223,"Baltimore, MD"
4,Calvert Memorial Hospital,h51968182,8,4,4,2.4,21.6,23.6,0.338983,1.66667,0.185185,1,"100 Hospital Rd, Prince Frederick, MD 20678, USA",Calvert County,Region V,Prince Frederick,20678,38.5594,-76.5963,ChIJGf93tYmCt4kRUHcfYxctVoM,Maryland,MD,21025,"Prince Frederick, MD",113,"Washington, DC"
5,Carroll Hospital Center,h73144964,16,4,12,4.0,44.8,49.2,0.325203,1.0,0.267857,1,"200 Memorial Ave, Westminster, MD 21157, USA",Carroll County,Region III,Westminster,21157,39.5576,-76.9903,ChIJX6DoR2s4yIkR_eqhMva6JnY,Maryland,MD,21033,"Westminster, MD",223,"Baltimore, MD"
6,Charles Regional (UM),h61059656,7,2,5,4.8,39.2,44.4,0.157658,0.416667,0.127551,1,"5 Garrett Ave, La Plata, MD 20646, USA",Charles County,Region V,La Plata,20646,38.529,-76.9722,ChIJL0cYbucIt4kR8Bh1L4nlJtc,Maryland,MD,21019,"La Plata, MD",113,"Washington, DC"
7,Chestertown (UMSRH),h95946725,0,0,0,0.8,4.4,5.2,0.0,0.0,0.0,1,"223 High St, Chestertown, MD 21620, USA",Kent County,Region IV,Chestertown,21620,39.2085,-76.0657,ChIJ_1Qaz0OLx4kRb8E80Y10zW0,Maryland,MD,21005,"Chestertown, MD",223,"Baltimore, MD"
8,Doctors Community Hospital,h82233897,52,13,39,8.8,74.8,83.6,0.62201,1.47727,0.52139,1,"8118 Good Luck Rd, Lanham, MD 20706, USA",Prince Georges County,Region V,Lanham,20706,38.9823,-76.865,ChIJlxkz022lt4kR4BGhrREiUM0,Maryland,MD,21020,"Lanham, MD",226,"Takoma Park, MD"
9,Dorchester (UMSRH),h63139553,0,0,0,0.0,3.6,3.6,0.0,0.0,0.0,1,"300 Byrn St, Cambridge, MD 21613, USA",Dorchester County,Region IV,Cambridge,21613,38.5714,-76.0674,ChIJW8KAmPZJuIkReFPCqJWGcWw,Maryland,MD,21004,"Cambridge, MD",225,"Salisbury, MD"


In [17]:
data_combined |> CSV.write("../data/current_load_covid.csv");

In [18]:
data_combined_list = collect([NamedTuple(h) for h in eachrow(data_combined)]);

In [19]:
serialize("../data/current_load_covid.jlser", data_combined_list)